# Update Raven HRUs

This script is used to update the Raven HRU definition file (*.rvh) according to a given land use map. This script would be run should the user wish to alter land use patterns and compare model outputs to the baseline conditions and asses how land use change can potentially affect local hydrology.

After completion of this script, lines are printed that are to be copy-and-pasted into the the Raven *HRU/Basin Definition file* (.rvh).

The simplest way to use this file is to alter the land use raster file name and click the `Run All` button. A text files with the necessary lines needed to be pasted into the model .rvh file will be created.

## Load Python packages and import data

In [1]:
import os
import math
import numpy as np
import shapefile
from pyproj import Proj
from PIL import Image, ImageDraw

### Import data, set user settings

This script requires 4 datasets + 1 user setting:

1. The sub-basin polygon shapefile--*this must <ins>not</ins> be modified for this script to work*
1. A digital elevation model (DEM)--*it is unlikely that this will need changing*
1. Surficial geology mapping--*this too will unlikely need changing*
1. Land use mapping--*This is the target raster that would need changing for the execution of this script*
1. minHRUfraction--*more on the purpose of the parameter below, can be changed but the value of 0.05 was maintained throughout the 2025 water balance study*

> All of the above files must be altered (where necessary) external, and prior to the execution, of this script. These input files are standard GIS files and should be altered by GIS professionals.

#### Files paths to alter
Land us cover--details of raster specifications are given below

In [2]:
lufp = '../GIS/CH_Landuse_dissolved.bil' # land use raster

#### File paths and Parameters that may be altered
inputs not likely/recommended to change

In [3]:
demfp = '../GIS/PDEM-South-D2013-OWRC23-60_resmpl.bil' # digital elevation model
sgfp = '../GIS/surfgeo23_60_resmpl.bil' # surficial geology raster

Script parameters (more details below)

In [4]:
minHRUfraction = 0.05 # recommended to be kept at 0.05

#### File path <ins>not</ins> to change
> Doing so may results in Raven model errors

In [5]:
sbasfp = '../GIS/CH_Subwatershed_select.shp' # modified CH sub-watershed boundaries

### Notes:
Please note that all input rasters must conform to th following pre-determined raster definition and projection.

#### Projection:

[EPSG:3161 - NAD83/Ontario MNR Lambert](https://epsg.io/3161):

```
PROJCS["NAD83 / Ontario MNR Lambert",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101],
            TOWGS84[0,0,0,0,0,0,0]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-85],
    PARAMETER["standard_parallel_1",44.5],
    PARAMETER["standard_parallel_2",53.5],
    PARAMETER["false_easting",930000],
    PARAMETER["false_northing",6430000],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","3161"]]
```

#### Raster Grid Definition:

[ESRI Band interleaved by line files (_*.bil_)](https://desktop.arcgis.com/en/arcmap/latest/manage-data/raster-and-images/bil-bip-and-bsq-raster-files.htm) are used in this script. These binary raster files must be accompanied with a grid definition header file (_*.hdr_) with the following spatial attributes:

```
NROWS          719
NCOLS          726
ULXMAP         1323330
ULYMAP         11906070
XDIM           60
YDIM           60
NODATA         -9999
```

In [6]:
# (keep consistent with input raster definition)
nrows = 719
ncols = 726
xul = 1323330
yul = 11906070
cs = 60. # grid cell width
epsg=3161

### Read data

#### Functions

The following are a set of functions needed to process the input data, *please do not alter these*. (Note that grid definition specification has been hard-coded into the functions.)

In [7]:
# Reads integer rasters
def readIntRaster(fp):
    aa = np.fromfile(fp,np.int32)
    if len(aa) == nrows*ncols:
        x = dict(zip(np.arange(nrows*ncols),aa))
    elif len(aa) == nrows*ncols/2:
        aa = np.fromfile(fp,np.int16)
        x = dict(zip(np.arange(nrows*ncols),aa))
    else:
        print(" ** Warning, unknown raster size {}",fp)
        return None
    return x

# Reads real-number rasters
def readFloatRaster(fp):
    aa = np.fromfile(fp,np.float32)
    aa[aa<-9999] = -9999
    if len(aa) == nrows*ncols:
        x = dict(zip(np.arange(nrows*ncols),aa))
    else:
        print(" ** Warning, unknown raster size {}",fp)
        return None
    return x

# saves binary raster
def saveBinaryInt(fp, dat):
    fn, _ = os.path.splitext(fp)
    with open(fn+'.hdr', 'w') as f: # save header
        f.write('BYTEORDER      I\n')
        f.write('LAYOUT         BIL\n')
        f.write('NROWS          '+str(nrows)+'\n')
        f.write('NCOLS          '+str(ncols)+'\n')
        f.write('NBANDS         1\n')
        f.write('NBITS          32\n')
        f.write('BANDROWBYTES   '+str(nrows*4)+'\n')
        f.write('TOTALROWBYTES  '+str(nrows*4)+'\n')
        f.write('PIXELTYPE      SIGNEDINT32\n')
        f.write('ULXMAP         '+str(xul+cs/2)+'\n') # The x-axis map coordinate of the center of the upper-left pixel.
        f.write('ULYMAP         '+str(yul-cs/2)+'\n') # The y-axis map coordinate of the center of the upper-left pixel.
        f.write('XDIM           '+str(cs)+'\n')
        f.write('YDIM           '+str(cs)+'\n')
        f.write('NODATA         -9999\n')
    def crc(c): # cell to row-col
        i = int(cid/ncols)
        j = c - i*ncols
        return (i,j)

    if type(dat)==dict:
        a = np.full((nrows,ncols),-9999,dtype=np.int32)
        for cid,v in dat.items(): a[crc(cid)] = np.int32(v)
        if os.path.exists(fp): os.remove(fp)
        a.tofile(fp) # always saved in C-order (row-major)
    # elif type(dat)==np.ndarray:
    #     if os.path.exists(fp): os.remove(fp)            
    #     dat.tofile(fp)
    else:
        print('saveBinaryInt error, unsupported type:',type(dat))

In [8]:
# get cell slopes and aspect from DEM
def slopeAspectTarboton(x):
    #  ref: Tarboton D.G., 1997. A new method for the determination of flow directions and upslope areas in grid digital elevation models. Water Resources Research 33(2). p.309-319.
    #  triangular facets, ordered by steepest (assumes uniform cells)
    #  facets (slightly modified from Tarboton, 1997):
    #         \2|1/
    #         3\|/0
    #         --+--
    #         4/|\7
    #         /5|6\        
    re1 = [0, -1, -1, 0, 0, 1, 1, 0]
    ce1 = [1, 0, 0, -1, -1, 0, 0, 1]
    re2 = [-1, -1, -1, -1, 1, 1, 1, 1]
    ce2 = [1, 1, -1, -1, -1, -1, 1, 1]
    ac = [0., 1., 1., 2., 2., 3., 3., 4.]
    af = [1., -1., 1., -1., 1., -1., 1., -1.]
    atan1 = math.atan(1.)  #math.Atan(1.)
    hcw = math.sqrt(2 * cs**2)
    
    sxs, rgs = dict(), dict()
    for cid,e0 in x.items():
        sx, rx, kx = 0., -9999, -1
        for k in range(8):
            c1, c2 = cid+re1[k]*ncols+ce1[k], cid+re2[k]*ncols+ce2[k]
            if not c1 in x: continue
            if not c2 in x: continue
            e1, e2 = x[c1], x[c2]
            if e1==-9999 or e2==-9999: continue
            if e1 > e0 and e2 > e0: continue

            s1 = (e0 - e1) / cs
            s2 = (e1 - e2) / cs
            r = math.atan2(s2,s1)
            s = math.sqrt(s1**2 + s2**2)
            if r < 0:
                r = 0
                s = s1
            elif r > atan1:
                r = atan1
                s = (e0 - e2) / hcw

            if s > sx:
                sx = s
                rx = r
                kx = k
        
        if kx < 0:
            sxs[cid], rgs[cid] = 0., -9999. # unspecified
            continue

        rg = af[kx]*rx + ac[kx]*math.pi/2.
        if rg > math.pi: rg -= 2 * math.pi # [-pi,pi]

        sxs[cid], rgs[cid] = sx, rg

    return sxs, rgs

In [9]:
# Get raster grid cells within sub-watershed polygons
def polygonToCellIDs(pts):
    img = Image.new('L', (ncols, nrows), 0)
    offset = [xul, yul-nrows*cs]
    pixelpoly = [((x-offset[0])/cs, nrows-(y-offset[1])/cs) for x,y in pts]
    ImageDraw.Draw(img).polygon(pixelpoly, outline=1, fill=1)
    mask = np.array(img)
    return list(np.arange(nrows*ncols)[mask.reshape(nrows*ncols)>0])

def loadSWS(fp):
    sf = shapefile.Reader(fp)
    geom = sf.shapes()
    attr = sf.records()
    xr = dict()
    lak = dict()
    csws = dict()
    for i in range(len(geom)):
        a = attr[i]
        sid = int(a.SubId)
        lak[sid] = a.lake != 0
        pcids = polygonToCellIDs(geom[i].points)
        for pc in pcids:
            if pc in csws:
                csws[pc].append(sid)
            else:
                csws[pc] = [sid]
        xr[sid] = [int(pc) for pc in pcids]

    # the polygonToCellIDs function can cause overlap where 2 adjacent polygons are assigned the same cell
    for c, sids in csws.items():
        if len(sids)<2: continue
        cmin, smin = 2**63-1 , -1
        for sid in sids:
            if len(xr[sid]) < cmin:
                cmin = len(xr[sid])
                smin = sid
        for sid in sids:
            if sid==smin: continue
            xr[sid].remove(c)

    return xr, lak

In [10]:
# special function to report raster data ranges
def crange(x):
    vv = [v for _, v in x.items()]
    print("[{},{}]".format(min(vv),max(vv)))

#### Data

Below all of 

- DEM, land use, surficial geology are loaded 
- Slopes and aspected are computed
- Cell indexing of sub-basin is determined

In [11]:
# Get cell elevations, slopes and aspects, cross-referenced to cell ID
elevations = readFloatRaster(demfp)
slopes, aspects = slopeAspectTarboton(elevations) # Note: this function will return a "RuntimeWarning: overflow encountered in scalar power s = math.sqrt(s1**2 + s2**2)" Please ignore

crange(elevations)
crange(slopes)
crange(aspects)

[-9999.0,414.9754943847656]
[0.0,4.668233313305587]
[-9999.0,3.141592653589793]


In [12]:
# Sub-basins cross-referenced to cell ID, and flag whether the sub-basin is of lake/reservoir type
xsb, islak = loadSWS(sbasfp)

In [13]:
# Read land use and surficial geology, cross-referenced to cell ID
xlu = readIntRaster(lufp)
xsg = readIntRaster(sgfp)
crange(xlu)
crange(xsg)

[-9999,11]
[0,200]


## Set Properties to Land use and Surficial Geology

### Land use

Land use provided by Conservation Halton is categorized into 11 classes. From these, a land surface type and a canopy type are assigned.

> When creating a new land use map to model, the map <ins>must</ins> conform to these 11 classes.

| ID | Class description | Land surface type | Canopy type | 
|:---|:---|:---|:---|
1 | Forest | Forest | Deciduous | 
2 | Hedge Row | Short vegetation | Mixed vegetation | 
3 | Marsh | Wetland | Short vegetation | 
4 | Swamp | Swamp | Short vegetation | 
5 | Forested Swamp | Swamp | Mixed vegetation | 
6 | Water | Waterbody | Bare | 
7 | Agriculture | Agriculture | Short vegetation | 
8 | Undifferentiated | Short vegetation | Short vegetation | 
9 | Settlement and Developed Lands | Urban | Bare | 
10 | Transportation | Barren | Bare | 
11 | Extraction-Aggregate | noflow | Bare | 

In [14]:
# used to convert CH land use ID to surface/canopy type
chlu = {
        1: ("Forest", "Deciduous"), # Forest
        2: ("ShortVegetation", "MixedVegetation"), # Hedge Row
        3: ("Wetland", "ShortVegetation"), # Marsh
        4: ("Swamp", "ShortVegetation"), # Swamp
        5: ("Swamp", "MixedVegetation"), # Forested Swamp
        6: ("Waterbody", "Bare"), # Water
        7: ("Agriculture", "ShortVegetation"), # Agriculture
        8: ("ShortVegetation", "ShortVegetation"), # Undifferentiated
        9: ("Urban", "Bare"), # Settlement and Developed Lands
        10: ("Barren", "Bare"), # Transportation
        11: ("noflow", "Bare"), # Extraction-Aggregate
    }
def chToTuple(x):
    o = dict()
    for c,v in x.items(): 
        if v < 0:
            o[c] = chlu[8] # default unknown/unspecified to Undifferentiated
        else:
            o[c] = chlu[v]
    return o

### Surficial geology

Surficial geology was obtained from Ontario Geological Survey (OGS) Surficial geology of southern Ontario. Full description can be found [here](https://owrc.github.io/interpolants/interpolation/surfgeo.html) and the source metadata can be found [here](https://owrc.github.io/metadata/surfaces/surficial_geology.html).  The input surficial geology raster is organized by the geologic type, the following table shows the relationship between material type and the relative permeability attribute assigned by the OGS:

| Raster ID | Relative Permeability | Material type |
|:---|:---|:---|
| 0 | Variable | Waterbody |
| 10 | Low | Precambrian bedrock |
| 20 | Low | Bedrock-drift complex in Precambrian terrain |
| 21 | Low | Bedrock-drift complex in Precambrian terrain: Primarily till cover |
| 22 | Low | Bedrock-drift complex in Precambrian terrain: Primarily stratified drift cover |
| 30 | Variable | Sedimentary (Paleozoic) bedrock |
| 40 | Variable | Bedrock-drift complex in Paleozoic terrain |
| 41 | Variable | Bedrock-drift complex in Paleozoic terrain: Primarily till cover |
| 50 | Medium | Till (Diamicton) |
| 51 | Medium | Till: Silty sand to sand-textured till on Precambrian terrain |
| 52 | LowMedium | Till: Stone-poor, sandy silt to silty sand-textured till on Paleozoic terrain |
| 53 | Medium | Till: Stony, sandy silt to silty sand-textured till on Paleozoic terrain |
| 54 | Low | Till: Clay to silt-textured till (derived from glaciolacustrine deposits or shale) |
| 55 | Variable | Till: Undifferentiated older tills, may include stratified deposits |
| 60 | High | Ice-contact stratified deposits |
| 61 | High | Ice-contact stratified deposits: In moraines, eskers, kames and crevasse fills |
| 62 | High | Ice-contact stratified deposits: In subaquatic fans |
| 70 | High | Glaciofluvial deposits: Sandy deposits |
| 71 | High | Glaciofluvial deposits: Gravelly deposits |
| 72 | High | Glaciofluvial deposits |
| 80 | Low | Fine-textured glaciolacustrine deposits |
| 81 | Low | Fine-textured glaciolacustrine deposits: Massive to well laminated |
| 82 | Low | Fine-textured glaciolacustrine deposits: Interbedded silt and clay and gritty, pebbly flow till and rainout deposits |
| 90 | High | Coarse-textured glaciolacustrine deposits |
| 91 | High | Coarse-textured glaciolacustrine deposits: Deltaic deposits |
| 92 | High | Coarse-textured glaciolacustrine deposits: Littoral deposits |
| 93 | High | Coarse-textured glaciolacustrine deposits: Foreshore and basinal deposits |
| 120 | Fluvial | Older alluvial deposits |
| 130 | Low | Fine-textured lacustrine deposits |
| 140 | High | Coarse-textured lacustrine deposits |
| 142 | High | Coarse-textured lacustrine deposits: Littoral deposits |
| 143 | High | Coarse-textured lacustrine deposits: Foreshore and basinal deposits |
| 170 | MediumHigh | Eolian deposits |
| 190 | Fluvial | Modern alluvial deposits |
| 200 | Organic | Organic Deposits |
| 210 | Variable | Fill |


From the above table, this layer is subdivided into 8 classes based on permeability attribute:

| ID | Relative Permeability |
|:---|:---|
| 1 | Low |
| 2 | Low Medium |
| 3 | Medium |
| 4 | Medium High |
| 5 | High |
| 6 | Unknown/variable |
| 7 | Streambed/fluvial/floodplain |
| 8 | Wetland sediments/organics |

In [15]:
# used to convert OGS ID to relative permeability
permlu = {
        1: 'Low',       
        2: 'LowMedium',
        3: 'Medium',
        4: 'MediumHigh',
        5: 'High',
        6: 'Unknown',
        7: 'Streambed',
        8: 'WetlandSediments',
    }
def ogsToRelPerm(x):
    o = dict()
    for c,v in x.items():
        match v:
            case 10 | 20 | 21 | 22 | 54 | 80 | 81 | 82 | 130:
                o[c] = permlu[1] # 'Low'
            case 52:
                o[c] = permlu[2] # 'LowMedium'
            case 50 | 51 | 53:
                o[c] = permlu[3] # 'Medium'
            case 170:
                o[c] = permlu[4] # 'MediumHigh'
            case 60 | 61 | 62 | 70 | 71 | 72 | 90 | 91 | 92 | 93 | 140 | 142 | 143:
                o[c] = permlu[5] # 'High'
            case 0 | 30 | 40 | 41 | 55 | 210:
                o[c] = permlu[6] # 'Variable'
            case 120 | 190:
                o[c] = permlu[7] # 'Fluvial'
            case 200:
                o[c] = permlu[8] # 'Organic'
            case _:
                print(' WARNING: no SG data for type {}'.format(v))
    return o

In [16]:
# update integer refernce to character reference for both land use and soil types
xlu = chToTuple(xlu)
xsg = ogsToRelPerm(xsg)

### Raster Overlay Grouping

The next step is to perform a raster overlay exercise to group combinations land use, surficial geology and sub-basin ID; it is from these groupings where Raven HRUs are derived. In order to optimize model runtimes, it is a good idea to aggregate those groupings that cover small portions of a sub-basin. Here, we declare that any group proportionally covering a sub-basin that is less than the user-defined parameter `minHRUfraction` be aggregated. 

> For the Conservations Halton 2025 water budget, `minHRUfraction` is set to 5%.

The aggregation of small-proportion combinations is then broken up into a maximum of 4 distinct land use classes covering an assumed relative permeability class:

1. Urban areas (ID: 9; variable permeability: 6)
1. Wetlands (ID: 3--5; Wetland sediments: 8)
1. Agriculture (ID: 7; variable permeability: 6)
1. Natural (ID: all else; variable permeability: 6)

#### Funtions

In [17]:
def buildHRUs(sbc, slak, lu, sg, zc, gc, ac):
    prj = Proj('EPSG:'+str(epsg))
    hrus = dict() # swsid: (lu,sg): fractional cover
    llzga = dict() # swsid: (lu,sg): (lat,long,elev,slope-angle,aspect)
    cells = dict() # cellid: (lu,sg)
    
    cc = 0
    for sid,cids in sbc.items():
        dt, xt, yt, zt, gt, axt, ayt, gnt = dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()
        n = len(cids)
        for c in cids:

            # HARD-CODED DEFAULTS
            ll = chlu[8]
            gg = permlu[6]
            
            if c in lu: ll = lu[c]
            if c in sg: gg = sg[c]
            t = (ll,gg)
            cells[c] = t

            # collect elevations and centroids per hru
            ii = int(c/ncols)
            jj = c - ii*ncols
            if zc[c]>-999:
                if t in dt: 
                    dt[t] += 1 
                    xt[t] += xul+(jj+0.5)*cs
                    yt[t] += yul-(ii+0.5)*cs
                    zt[t] += zc[c] # sum elevation
                else: 
                    dt[t] = 1
                    xt[t] = xul+(jj+0.5)*cs
                    yt[t] = yul-(ii+0.5)*cs          
                    zt[t] = zc[c]

            # collect slopes and aspects per hru
            if gc[c] > 0:
                if t in gt:
                    gnt[t] += 1
                    gt[t] += math.atan(gc[c]) # sum slopes (convert rise/run to angle)
                    axt[t] += math.sin(ac[c]) # sum x-component of aspect
                    ayt[t] += math.cos(ac[c]) # sum y-component of aspect
                else:
                    gnt[t] = 1
                    gt[t] = math.atan(gc[c])
                    axt[t] = math.sin(ac[c])
                    ayt[t] = math.cos(ac[c])

        if slak[sid]: # setting sub-basin as a lake HRU
            hrus[sid] = "lake"
            sx, sy, sz, sd = 0., 0., 0., 0.
            for t,dd in dt.items():
                sd += dd
                sx += xt[t]
                sy += yt[t]
                sz += zt[t]

            lng,lat = prj(sx/sd, sy/sd, inverse=True)
            llzga[sid] = (lat,lng,float(sz/sd))
            cc+=1
        else:
            dd, sa = dict(), dict()
            x, y, z, g, ax, ay, gn = dict(), dict(), dict(), dict(), dict(), dict(), dict()
            def aggregate(t,tg,v,cidm=None):                            
                if tg in dd:
                    dd[tg] += v
                    x[tg] += xt[t]
                    y[tg] += yt[t]
                    z[tg] += zt[t]
                    if t in gnt and gnt[t]>0:
                        g[tg] += gt[t]
                        gn[tg] += gnt[t]
                        ax[tg] += axt[t]
                        ay[tg] += ayt[t]
                else:
                    dd[tg] = v
                    x[tg] = xt[t]
                    y[tg] = yt[t]
                    z[tg] = zt[t]
                    if t in gnt and gnt[t]>0:
                        g[tg] = gt[t]
                        gn[tg] = gnt[t]
                        ax[tg] = axt[t]
                        ay[tg] = ayt[t]
                if cidm is not None:
                    for c in cidm: cells[c]=tg  

            for t,v in dt.items():
                if v/n > minHRUfraction: # large combination, keep as-is
                    aggregate(t,t,v)
                else:                    # small combination, aggregate to general groups
                    cidm = list()
                    for c in cids: 
                        if cells[c]==t: cidm.append(c)          
                    match t[0]:
                        case 9: # urban
                            aggregate(t,(chlu[9],permlu[6]),v,cidm)
                        case 7: # agriculture
                            aggregate(t,(chlu[7],permlu[6]),v,cidm)
                        case 3 | 4 | 5: # wetland
                            aggregate(t,(chlu[3],permlu[8]),v,cidm)
                        case _: # other/natural
                            aggregate(t,(chlu[8],permlu[6]),v,cidm)
            
            for t,v in dd.items(): 
                dd[t]/=n # normalize
                lng,lat = prj(x[t]/v, y[t]/v, inverse=True)
                if not t in gn or gn[t]==0:
                    sa[t] = (lat,lng,float(z[t]/v),0.,0.)
                else:
                    gg = g[t]/gn[t] # radians
                    aa = (math.atan2(ax[t]/gn[t],ay[t]/gn[t]) + 2 * np.pi) % (2 * np.pi) # [0 to 2pi]
                    sa[t] = (lat,lng,float(z[t]/v),gg,aa)

            hrus[sid] = dd
            llzga[sid] = sa
            cc += len(dd)

    print(' {} distinct HRUs'.format(cc))

    return hrus, llzga, cells  

In [18]:
def printHRUIDs(fp, sbc, hrus, cells):
    d = dict()
    tt = dict()
    c = 0
    for sid,lusg in hrus.items():
        if lusg=='lake':
            c+=1
            for cid in sbc[sid]: d[cid] = c

    for sid,lusg in hrus.items():
        if lusg=='lake': continue
        tt[sid] = dict()
        for t,_ in lusg.items():
            c+=1
            tt[sid][t] = c

    for sid,cids in sbc.items():
        if not sid in tt: continue
        for c in cids:
            t = cells[c]
            d[c] = tt[sid][t]

    saveBinaryInt(fp,d)

#### Processing

Build HRUs and output an HRU ID raster for later reference (i.e., the *output_to_raster.ipynb* script)

In [19]:
hrus, llzga, cells = buildHRUs(xsb,islak,xlu,xsg,elevations,slopes,aspects)
printHRUIDs(lufp[:-4]+'-hruid.bil',xsb,hrus,cells) # Save HRU ID as raster

 420 distinct HRUs


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '../GIS/CH_Landuse_dissolved-hruid.bil'

## Printing Raven HRU specifications

Print the HRU lines to be copied into the _*.rvh_ file

In [ ]:
def rad2deg(rad): return (rad/math.pi*180.) % 360 # note Raven aspect is specified as degrees Counter-Clockwise from the North (i.e., west=90°)
def printRVH(f,ln):
    print(ln)
    f.write(ln+'\n')

In [ ]:
f = open(lufp[:-4]+'-rvh.txt', "w")
print('Writing to: '+lufp[:-4]+'-rvh.txt\n')

printRVH(f,':HRUs')
printRVH(f,' :Attributes      AREA ELEVATION  LATITUDE LONGITUDE  BASIN_ID      LAND_USE_CLASS           VEG_CLASS        SOIL_PROFILE AQUIFER_PROFILE  TERRAIN_CLASS     SLOPE    ASPECT') # aspect deg CCWN (west=90°)
printRVH(f,' :Units            km2         m       deg       deg      none                none                none                none            none           none       deg      degN')
c = 0

for sid,lusg in hrus.items(): # write lake hrus first
    if lusg=='lake':
        ll = llzga[sid]
        km2 = len(xsb[sid])*cs*cs/1000000
        c+=1
        printRVH(f,'  {:<10}{:10.4f}{:10.1f}{:10.4f}{:10.4f}{:10}{:>20}{:>20}{:>20}          [NONE]         [NONE]{:10.3f}{:10.3f}'.format(c,km2,ll[2],ll[0],ll[1],sid,'LAKE','LAKE','LAKE',0.,0.))

for sid,lusg in hrus.items():
    if lusg=='lake': continue
    ll = llzga[sid]
    km2 = len(xsb[sid])*cs*cs/1000000
    for k,frac in lusg.items():
        c += 1
        printRVH(f,'  {:<10}{:10.4f}{:10.1f}{:10.4f}{:10.4f}{:10}{:>20}{:>20}{:>20}          [NONE]         [NONE]{:10.3f}{:10.3f}'.format(c,km2*frac,ll[k][2],ll[k][0],ll[k][1],sid,k[0][0],k[0][1],k[1],rad2deg(ll[k][3]),rad2deg(ll[k][4]-math.pi/2)))

printRVH(f,':EndHRUs')
f.close()

Writing to: ../GIS/CH_Landuse_dissolved-rvh.txt

:HRUs
 :Attributes      AREA ELEVATION  LATITUDE LONGITUDE  BASIN_ID      LAND_USE_CLASS           VEG_CLASS        SOIL_PROFILE AQUIFER_PROFILE  TERRAIN_CLASS     SLOPE    ASPECT
 :Units            km2         m       deg       deg      none                none                none                none            none           none       deg      degN
  1             2.2212     298.3   43.4603  -80.0442        64                LAKE                LAKE                LAKE          [NONE]         [NONE]     0.000     0.000
  2             0.1944     281.0   43.5105  -79.9661        67                LAKE                LAKE                LAKE          [NONE]         [NONE]     0.000     0.000
  3             0.3816     230.2   43.5100  -79.9467        65                LAKE                LAKE                LAKE          [NONE]         [NONE]     0.000     0.000
  4             0.4212     252.3   43.5727  -79.9480        69             